In [2]:
while (True):
    context = input("Input context: \n")
    if (context.lower() == "ok"):
        break
    answer = input("Input Answer: \n")
    if (answer.lower() == "ok"):
        break
    
    print("Start is: ", context.find(answer))

Start is:  225


In [20]:
import re
from underthesea import word_tokenize
from rank_bm25 import BM25Okapi, BM25Plus
import pickle
from datasets import Dataset
import json
import torch
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering, DefaultDataCollator, TrainingArguments, Trainer, AutoModelForQuestionAnswering

In [21]:
# Function read file train
def read_data(filePath):
    f = open(filePath,encoding='utf-8')
    fileRead = json.load(f)
    context = []
    question = []
    answer = []
    for object in fileRead:
        for qa in object["qas"]:
            context.append(object["context"].lower())
            question.append(qa["question"].lower())
            answer.append({
                "text": [qa["answer"]["text"][0].lower()],
                "start": [qa["answer"]["start"][0]]
            })
    dataset = {
    "context": context,
    "question": question,
    "answer": answer
    }
    return dataset

# Function read file test
def read_data_test(filePath):
    f = open(filePath,encoding='utf-8')
    fileRead = json.load(f)
    question = []
    answer = []
    for object in fileRead:
        question.append(object["question"].lower())
        answer.append(object["answer"].lower())
    dataset = {
    "question": question,
    "answer": answer
    }
    return dataset

In [14]:
dataset = read_data("./QA_data/qa_train.json")
eval_dataset = read_data("./QA_data/qa_eval.json")

In [10]:
from datasets import Dataset
import pandas as pd

In [8]:
df = pd.DataFrame(dataset)

In [11]:
df

,context,question,answer
0,"bộ luật này quy định địa vị pháp lý, chuẩn mực...",quy định chung của bộ luật dân sự là gì ?,"{'text': ['quy định địa vị pháp lý, chuẩn mực ..."
1,"bộ luật này quy định địa vị pháp lý, chuẩn mực...",tôi muốn biết tổng quan về bộ luật dân sự.,"{'text': ['quy định địa vị pháp lý, chuẩn mực ..."
2,"bộ luật này quy định địa vị pháp lý, chuẩn mực...",bộ luật dân sự được hình thành dựa vào cái gì ?,{'text': ['được hình thành trên cơ sở bình đẳn...
3,"ở nước cộng hòa xã hội chủ nghĩa việt nam, các...",quyền dân sự được công nhận ở đâu ?,"{'text': ['quyền dân sự được công nhận, tôn tr..."
4,"ở nước cộng hòa xã hội chủ nghĩa việt nam, các...","quyền dân sự được công nhận, đảm bảo theo hiến...","{'text': ['quyền dân sự được công nhận, tôn tr..."
...,...,...,...
1114,người giám hộ được thay đổi trong trường hợp s...,muốn thay đổi người giám hộ có được không ?,{'text': ['được thay đổi trong trường hợp sau ...
1115,việc chuyển giao giám hộ phải được lập thành v...,chuyển giao người giám hộ có cần gì không ?,"{'text': ['phải được lập thành văn bản, trong ..."
1116,trường hợp chấm dứt việc giám hộ quy định tại ...,trong trường hợp nào việc giám hộ bị chấm dứt ?,{'text': ['trường hợp chấm dứt việc giám hộ qu...
1117,trường hợp chấm dứt việc giám hộ quy định tại ...,trường hợp nào việc giám hộ bị chấm dứt ?,{'text': ['trường hợp chấm dứt việc giám hộ qu...


In [15]:
df2 = pd.DataFrame(eval_dataset)

In [16]:
df2

,context,question,answer
0,"cá nhân, pháp nhân phải tự chịu trách nhiệm về...",cá nhân có chịu trách nhiệm khi trốn nghĩa vụ ...,"{'text': ['cá nhân, pháp nhân phải tự chịu trá..."
1,"cá nhân, pháp nhân phải tự chịu trách nhiệm về...",cá nhân có chịu trách nhiệm khi thực hiện khôn...,"{'text': ['cá nhân, pháp nhân phải tự chịu trá..."
2,trách nhiệm dân sự do vi phạm nghĩa vụ: bên có...,có chịu trách nhiệm khi thực hiện không đúng n...,{'text': ['bên có nghĩa vụ mà vi phạm nghĩa vụ...
3,luật khác có liên quan điều chỉnh quan hệ dân ...,luật khác có liên quan đến điều chỉnh quan hệ ...,{'text': ['điều chỉnh quan hệ dân sự trong các...
4,trường hợp luật khác có liên quan không quy đị...,bộ luật khác ảnh hưởng đến khoản 2 điều 4 thì ...,{'text': ['quy định của bộ luật này được áp dụ...
...,...,...,...
180,bộ luật này có hiệu lực thi hành từ ngày 01 th...,bộ luật này được xử dụng vào năm nào ?,{'text': ['bộ luật này có hiệu lực thi hành từ...
181,bộ luật này có hiệu lực thi hành từ ngày 01 th...,bộ luật này có hiệu lực từ bao giờ ?,{'text': ['bộ luật này có hiệu lực thi hành từ...
182,bộ luật này có hiệu lực thi hành từ ngày 01 th...,bộ luật dân sự có hiệu lực từ bao giờ ?,{'text': ['bộ luật này có hiệu lực thi hành từ...
183,bộ luật này là luật chung điều chỉnh các quan ...,luật dân sự điều chỉnh những quan hệ nào ?,"{'text': ['điều chỉnh các quan hệ dân sự'], 's..."


In [18]:
# To get score predict with bleu
test_data = read_data_test("./QA_data/qa_test.json")
df3= pd.DataFrame(test_data)

In [26]:
model = RobertaForQuestionAnswering.from_pretrained("vinai/phobert-base")
tokenizer = RobertaTokenizerFast.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a

In [27]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=128,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["start"][0]
        end_char = answer["start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [28]:
dataset_train = Dataset.from_dict(dataset)
dataset_eval = Dataset.from_dict(eval_dataset)

In [29]:
tokenized_squad = dataset_train.map(preprocess_function, batched=True, remove_columns=dataset_train.column_names)
tokenized_squad_eval = dataset_eval.map(preprocess_function, batched=True, remove_columns=dataset_eval.column_names)

In [31]:
dfs = pd.DataFrame(data=tokenized_squad, columns=tokenized_squad.column_names)

In [32]:
dfs

,input_ids,attention_mask,start_positions,end_positions
0,"[0, 4026, 2608, 417, 7, 215, 1252, 452, 61, 8,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",17,62
1,"[0, 70, 202, 55, 698, 2665, 28, 215, 1252, 452...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",17,62
2,"[0, 215, 1252, 452, 61, 11, 819, 302, 1054, 33...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",64,90
3,"[0, 493, 452, 61, 11, 675, 154, 25, 602, 114, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",25,48
4,"[0, 493, 452, 61, 11, 675, 39451, 1301, 4, 157...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",32,55
...,...,...,...,...
1114,"[0, 202, 1016, 1361, 18, 13466, 506, 10, 11, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",16,127
1115,"[0, 430, 574, 18, 13466, 506, 10, 115, 148, 17...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",18,81
1116,"[0, 12, 212, 2288, 142, 49, 13466, 506, 45, 24...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",14,36
1117,"[0, 212, 2288, 142, 49, 13466, 506, 45, 2475, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",13,35
